In [1]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import IPython as ip

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import yacht_main as yacht
from yacht_test import create_train_set

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
torch.cuda.is_available()

True

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """transition 저장"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
INPUT_SIZE = 19
OUTPUT_SIZE = 12

class DQN(nn.Module):

    def __init__(self, input_size, output_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 300)
        self.fc2 = nn.Linear(300, 300)
        self.fc3 = nn.Linear(300, 50)
        #self.fc4 = nn.Linear(100, 100)
        #self.fc5 = nn.Linear(100, 100)
        self.fc6 = nn.Linear(50, output_size)
        
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        #x = F.relu(self.fc4(x))
        #x = F.relu(self.fc5(x))

        x = self.fc6(x)

        return x

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m,0.1)


policy_net = DQN(INPUT_SIZE,OUTPUT_SIZE).to(device)
target_net = DQN(INPUT_SIZE,OUTPUT_SIZE).to(device)
#W = torch.tensor((43,1), device = device, dtype = torch.float)

init_weights(policy_net)
target_net.load_state_dict(policy_net.state_dict())
#init_weights(W)
#policy_net.apply(init_weights)
#target_net.apply(init_weights)

<All keys matched successfully>

In [4]:
BATCH_SIZE = 128
MEMORY_SIZE = 10000
GAMMA = 0.8
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 500
TARGET_UPDATE = 5
ALPHA = 0.1
REWARD_SCALE = 1.0/50
TRAINSET_PERIOD = 1000

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(MEMORY_SIZE)


steps_done = 0


def select_action(state, avail):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            # 최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            # 기대 보상이 더 큰 행동을 선택할 수 있습니다.
            return (policy_net(state) * avail).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(OUTPUT_SIZE)]], device=device, dtype=torch.long)



episode_scores = []


def plot_scores():
    plt.figure(2)
    plt.clf()
    scores_t = torch.tensor(episode_scores, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.plot(scores_t.numpy())
    # 100개의 에피소드 평균을 가져 와서 도표 그리기
    if len(scores_t) >= 50:
        means = scores_t.unfold(0, 50, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(49), means))
        plt.plot(means.numpy())

    ip.display.clear_output(wait=True)
    plt.pause(0.001)  # 도표가 업데이트되도록 잠시 멈춤
    #if is_ipython:
    #display(plt.gcf())

In [5]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [6]:
def add_trainset():
    print("Creating train set...")
    train_set_size = 3000 // 5
    train_set = create_train_set(train_set_size)
    for state, action, new_state, step_reward in train_set:
        state_tensor = torch.tensor(state, dtype=torch.float, device=device, requires_grad = False)
        action_tensor = torch.tensor([[action]], device=device, dtype=torch.long)
        new_state_tensor = torch.tensor(new_state, dtype=torch.float, device=device, requires_grad = False)
        step_reward_tensor = torch.tensor([step_reward], device=device, requires_grad = False)
        memory.push(state_tensor.reshape(1,INPUT_SIZE), action_tensor, new_state_tensor.reshape(1,INPUT_SIZE), step_reward_tensor)
    print("Created", train_set_size * 5, "train set")

# num_episodes = 500000
for i_episode in range(num_episodes):
    # 환경과 상태 초기화
    yacht.reset_game()
    state, score, _, avail = yacht.get_yacht_output()
    state = torch.tensor(state, dtype=torch.float, device=device, requires_grad = False)
    for t in count():
        # 행동 선택과 수행
        action = select_action(state.reshape(1,INPUT_SIZE), \
                               torch.tensor(avail, dtype=torch.float, device=device, requires_grad = False).reshape(1, OUTPUT_SIZE))
        yacht.update(action)
        
        new_state, new_score, done, avail = yacht.get_yacht_output()
        step_reward = torch.tensor([new_score - score], device=device, requires_grad = False)

        if not done:
            new_state = torch.tensor(new_state, dtype=torch.float, device=device, requires_grad = False)
            memory.push(state.reshape(1,INPUT_SIZE), action, new_state.reshape(1,INPUT_SIZE), \
                    step_reward)
        else:
            new_state = None
            memory.push(state.reshape(1,INPUT_SIZE), action, None, \
                    step_reward)
        

        # 다음 상태로 이동
        state = new_state
        score = new_score

        # 최적화 한단계 수행(목표 네트워크에서)
        optimize_model()
        if done:
            state, score, _, _ = yacht.get_yacht_output()
            episode_scores.append(score)
            #print("{0}) {1}\tscore : {2}, turns = {3}".format(i_episode, state[:12], score, t+1))
            
            if i_episode % 200 == 0:
                GAMMA+=0.01
                plot_scores()
            
            break
        
    #목표 네트워크 업데이트, 모든 웨이트와 바이어스 복사
    if i_episode % TRAINSET_PERIOD == 0:
        add_trainset()
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    if i_episode % 10000 == 0:
        torch.save(policy_net.state_dict(), './data/solo_net_' + str(i_episode//10000))

print('Complete')

In [ ]:
def step_by_step():
    yacht.reset_game()
    
    state, _, _ = yacht.get_yacht_output()
    print("{0}\tscore : {1}, turns = {2}".format(state, 0, 0))
    state = torch.tensor(state, dtype=torch.float, device=device, requires_grad = False)
    
    for t in count():
#        policy_net(state).max(1)[1].view(1, 1)
        action = select_action(state.reshape(1,INPUT_SIZE))
        print("\nActions: {0}".format(action[0][0] ))
        yacht.update(action)
        
        state, score, done = yacht.get_yacht_output()
        print("{0}\tscore : {1}, turns = {2}".format(state, score, t+1))
        state = torch.tensor(state, dtype=torch.float, device=device, requires_grad = False)
        
        if done:
            break
step_by_step()

In [ ]:
def print_memory(mem):
    print("\n\nState: {0}".format(mem.state[0].int().tolist() ))
    print("Action: {0}".format(action[0][0] ))
    if mem.next_state == None:
        print("State: None")
    else:
        print("State: {0}".format(mem.next_state[0].int().tolist() ))
    print("Reward: {0}".format(mem.reward[0][0].int() ))
    

In [ ]:
for i in range(10):
    print_memory(memory.memory[i])

In [ ]:
[[-1]*12]*2

In [ ]:
test_net = DQN(INPUT_SIZE, OUTPUT_SIZE)
test_net.load_state_dict(torch.load('./data/solo_net_2'))
test_net.eval()

In [ ]:
yacht.reset_game()

In [ ]:
state,reward,_,avail = yacht.get_yacht_output()
print(state)
midaction = test_net(torch.tensor(state, dtype=torch.float).reshape(1,INPUT_SIZE))
print(midaction)
print(avail)
action = (midaction * torch.tensor(avail, dtype=torch.float).reshape(1,OUTPUT_SIZE)).max(1)[1]
yacht.update(action)
print(action-30)
state,reward,_, _ = yacht.get_yacht_output()
print(state)

In [ ]:
device

In [ ]:
torch.ones(BATCH_SIZE, device=device) * -100